In [2]:
import mysql.connector
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt 
import koreanize_matplotlib
from plotly.subplots import make_subplots
import os, json
import pandas as pd

In [3]:
aws_RDS = mysql.connector.connect(
host = "database-1.cz44qgu84nin.ap-northeast-2.rds.amazonaws.com",
user = "jb",
password = "1234",
database = "edabase"
)

In [4]:
crime = aws_RDS.cursor(dictionary=True, buffered=True)

In [31]:
crime.execute("SELECT * FROM crime_age_after")
result = crime.fetchall()

In [32]:
crime_age_df = pd.DataFrame(result)
crime_age_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,총범죄,소계,소계,미성년,86616,104532,88549,77280,79342,75757,72425,65784,65907,64152,53760,60691
1,총범죄,소계,소계,14세미만,316,654,281,54,80,78,88,0,0,0,0,0
2,총범죄,소계,소계,성인,1728590,1618724,1651378,1633101,1689960,1768887,1611388,1513619,1516477,1425704,1192060,1187696
3,총범죄,소계,소계,미상,27,559,1375,2054,2088,2961,1648,2519,3254,4565,1860,1943
4,총범죄,강력범죄,소계,미성년,3209,3231,3052,2610,2377,2410,2311,2267,2358,1898,1648,1808


In [33]:
year_list = ["2011년", "2012년", "2013년", "2014년", "2015년", "2016년", "2017년", "2018년", "2019년", "2020년", "2021년", "2022년"]
column_list = ["대", "중", "소", "연령별", "|2011년", "2012년", "2013년", "2014년", "2015년", "2016년" "2017년", "2018년" "2019년", "2020년", "2021년", "2022년"]

In [34]:
crime_age_df = crime_age_df.rename(columns={0:"대", 1:"중", 2:"소", 3:"연령별", 4:"2011년", 5:"2012년", 6:"2013년", 7:"2014년", 8:"2015년", 9:"2016년", 10:"2017년", 11:"2018년", 12:"2019년", 13:"2020년", 14:"2021년", 15:"2022년"}, errors='raise', copy=True)
crime_age_df.head()

,대,중,소,연령별,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,총범죄,소계,소계,미성년,86616,104532,88549,77280,79342,75757,72425,65784,65907,64152,53760,60691
1,총범죄,소계,소계,14세미만,316,654,281,54,80,78,88,0,0,0,0,0
2,총범죄,소계,소계,성인,1728590,1618724,1651378,1633101,1689960,1768887,1611388,1513619,1516477,1425704,1192060,1187696
3,총범죄,소계,소계,미상,27,559,1375,2054,2088,2961,1648,2519,3254,4565,1860,1943
4,총범죄,강력범죄,소계,미성년,3209,3231,3052,2610,2377,2410,2311,2267,2358,1898,1648,1808


In [35]:
theft_age_df = crime_age_df[crime_age_df["중"] == '절도범죄']
theft_age_df.head()

,대,중,소,연령별,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
44,총범죄,절도범죄,소계,미성년,32693,36634,32499,25999,25914,22507,19971,16873,17111,17027,12692,14772
45,총범죄,절도범죄,소계,14세미만,63,276,69,5,13,5,8,0,0,0,0,0
46,총범죄,절도범죄,소계,성인,78695,67779,70136,69613,77210,83833,85681,80381,84099,81328,72931,79478
47,총범죄,절도범죄,소계,미상,2,21,23,33,42,75,43,80,85,70,64,65


In [36]:
theft_age_df.set_index("연령별", inplace=True)
theft_age_df.drop(["대", "중", "소"], axis=1, inplace=True)
theft_age_df = theft_age_df.transpose()
theft_age_df

/tmp/ipykernel_19617/1925795677.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



연령별,미성년,14세미만,성인,미상
2011년,32693,63,78695,2
2012년,36634,276,67779,21
2013년,32499,69,70136,23
2014년,25999,5,69613,33
2015년,25914,13,77210,42
2016년,22507,5,83833,75
2017년,19971,8,85681,43
2018년,16873,0,80381,80
2019년,17111,0,84099,85
2020년,17027,0,81328,70


In [108]:
theft_age_df.index

Index(['2011년', '2012년', '2013년', '2014년', '2015년', '2016년', '2017년', '2018년',
       '2019년', '2020년', '2021년', '2022년'],
      dtype='object')

In [113]:
theft_age_df.columns

Index(['미성년', '14세미만', '성인', '미상', '년도'], dtype='object', name='연령별')

In [131]:
fig = go.Figure()

fig.update_layout(title_text="년도별 나이 대에 따른 절도범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22)
fig.update_xaxes(title_text="년도")
fig.update_yaxes(title_text="발생 건수")
fig.add_trace(go.Scatter(x=theft_age_df.index, y=theft_age_df["14세미만"], name='14세미만', mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_age_df.index, y=theft_age_df["미성년"], name='미성년',mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_age_df.index, y=theft_age_df["성인"], name='성인', mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_age_df.index, y=theft_age_df["미상"], name='미상', mode='lines+markers'))

fig.show()

In [37]:
vehicle_age_df = crime_age_df[crime_age_df["중"]=="교통범죄"]
vehicle_age_df

,대,중,소,연령별,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
152,총범죄,교통범죄,소계,미성년,11795,11977,9751,9247,9022,9307,9338,7391,7081,7779,6684,6611
153,총범죄,교통범죄,소계,14세미만,123,42,31,6,13,10,23,0,0,0,0,0
154,총범죄,교통범죄,소계,성인,512553,500039,526907,523635,540135,545644,475077,405659,376521,348544,304493,233527
155,총범죄,교통범죄,소계,미상,2,60,98,244,217,401,137,97,115,123,55,78


In [38]:
vehicle_age_df.set_index("연령별", inplace=True)
vehicle_age_df.drop(["대", "중", "소"], axis=1, inplace=True)
vehicle_age_df = vehicle_age_df.transpose()
vehicle_age_df

/tmp/ipykernel_19617/1926953815.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



연령별,미성년,14세미만,성인,미상
2011년,11795,123,512553,2
2012년,11977,42,500039,60
2013년,9751,31,526907,98
2014년,9247,6,523635,244
2015년,9022,13,540135,217
2016년,9307,10,545644,401
2017년,9338,23,475077,137
2018년,7391,0,405659,97
2019년,7081,0,376521,115
2020년,7779,0,348544,123


In [39]:
fig = go.Figure()

fig.update_layout(title_text="2016년 후 나이 대에 따른 교통범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22 )
fig.update_xaxes(title_text="년도")
fig.update_yaxes(title_text="발생 건수")
fig.add_trace(go.Scatter(x=vehicle_age_df.index[5:], y=theft_age_df["14세미만"][5:], name='14세미만', mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_age_df.index[5:], y=theft_age_df["미성년"][5:], name='미성년',mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_age_df.index[5:], y=theft_age_df["성인"][5:], name='성인', mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_age_df.index[5:], y=theft_age_df["미상"][5:], name='미상', mode='lines+markers'))

fig.show()

In [17]:
crime.execute("SELECT * FROM crime_motive_after")
result = crime.fetchall()

crime_motive_df = pd.DataFrame(result)
crime_motive_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,소계,소계,소계,이욕,157196,140413,134375,125751,139683,149315,144996,135729,144146,137185,111711,110149
1,소계,소계,소계,사행심,26902,22190,20705,19462,18322,20396,16361,17034,18666,16916,12457,11839
2,소계,소계,소계,보복,345,336,339,347,371,496,349,297,305,324,379,366
3,소계,소계,소계,가정불화,5753,6069,7255,6977,19857,25730,22481,20314,22845,20135,23208,20357
4,소계,소계,소계,호기심,19546,16230,17336,16053,16177,21623,17666,9384,10375,13367,13438,13985


In [18]:
crime_motive_df = crime_motive_df.rename(columns={0:"대", 1:"중", 2:"소", 3:"연령별", 4:"2011년", 5:"2012년", 6:"2013년", 7:"2014년", 8:"2015년", 9:"2016년", 10:"2017년", 11:"2018년", 12:"2019년", 13:"2020년", 14:"2021년", 15:"2022년"}, errors='raise', copy=True)
crime_motive_df.head()

,대,중,소,연령별,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,소계,소계,소계,이욕,157196,140413,134375,125751,139683,149315,144996,135729,144146,137185,111711,110149
1,소계,소계,소계,사행심,26902,22190,20705,19462,18322,20396,16361,17034,18666,16916,12457,11839
2,소계,소계,소계,보복,345,336,339,347,371,496,349,297,305,324,379,366
3,소계,소계,소계,가정불화,5753,6069,7255,6977,19857,25730,22481,20314,22845,20135,23208,20357
4,소계,소계,소계,호기심,19546,16230,17336,16053,16177,21623,17666,9384,10375,13367,13438,13985


In [19]:
crime_motive_df = crime_motive_df.rename(columns={"연령별":"범행동기"}, errors='raise', copy=True)

theft_motive_df= crime_motive_df[crime_motive_df["중"]=="절도범죄"]
theft_motive_df.head()

,대,중,소,범행동기,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
121,소계,절도범죄,소계,이욕,31921,29688,29445,26909,30625,30853,30220,24897,25109,24558,22226,22698
122,소계,절도범죄,소계,사행심,2065,2023,1906,1650,1724,1600,1729,1729,1802,1854,1607,1926
123,소계,절도범죄,소계,보복,19,12,10,4,3,7,2,1,5,2,6,2
124,소계,절도범죄,소계,가정불화,84,74,90,77,67,71,69,29,26,35,40,44
125,소계,절도범죄,소계,호기심,6923,5583,5299,4813,4882,4680,4274,1438,1474,2799,2756,2924


In [20]:
theft_motive_df.set_index("범행동기", inplace=True)
theft_motive_df.drop(["대", "중", "소"], axis=1, inplace=True)
theft_motive_df = theft_motive_df.transpose()
theft_motive_df

/tmp/ipykernel_19617/1949608987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  theft_motive_df.drop(["대", "중", "소"], axis=1, inplace=True)


범행동기,이욕,사행심,보복,가정불화,호기심,유혹,우발적,현실불만,부주의,기타,미상
2011년,31921,2065,19,84,6923,3246,30259,472,2912,23289,10200
2012년,29688,2023,12,74,5583,3251,29104,304,2342,24599,7454
2013년,29445,1906,10,90,5299,3168,29030,356,2512,23511,7331
2014년,26909,1650,4,77,4813,2524,26840,272,2581,22556,7419
2015년,30625,1724,3,67,4882,2304,28057,275,2465,24416,8348
2016년,30853,1600,7,71,4680,2170,29117,295,3162,25513,8947
2017년,30220,1729,2,69,4274,2112,29487,278,3422,24155,9947
2018년,24897,1729,1,29,1438,1254,29707,184,3125,17838,17132
2019년,25109,1802,5,26,1474,1171,31717,194,3351,18243,18203
2020년,24558,1854,2,35,2799,1070,31334,180,3319,19012,14262


In [21]:
fig = go.Figure()

fig.update_layout(title_text="년도에 따른 범죄동기 별 절도범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22 )
fig.update_xaxes(title_text="년도")
fig.update_yaxes(title_text="발생 건수")
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["이욕"], name='이욕', mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["사행심"], name='사행심',mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["보복"], name='보복', mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["가정불화"], name='가정불화', mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["호기심"], name='호기심', mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["유혹"], name='유혹',mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["우발적"], name='우발적', mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["현실불만"], name='현실불만', mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["부주의"], name='부주의',mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["기타"], name='기타', mode='lines+markers'))
fig.add_trace(go.Scatter(x=theft_motive_df.index, y=theft_motive_df["미상"], name='미상', mode='lines+markers'))

fig.show()

In [24]:
vehicle_motive_df= crime_motive_df[crime_motive_df["중"]=="교통범죄"]
vehicle_motive_df

,대,중,소,범행동기,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
418,소계,교통범죄,소계,이욕,8251,7834,5529,1315,1089,1202,1146,992,652,603,592,585
419,소계,교통범죄,소계,사행심,187,76,196,189,199,136,89,233,255,254,251,196
420,소계,교통범죄,소계,보복,4,3,12,0,3,3,2,2,2,1,2,1
421,소계,교통범죄,소계,가정불화,233,174,209,192,162,160,120,88,89,86,74,76
422,소계,교통범죄,소계,호기심,1490,1535,1587,1387,1338,1622,1712,606,701,1426,1691,1546
423,소계,교통범죄,소계,유혹,2082,1767,1293,876,1300,1350,1198,730,469,488,408,464
424,소계,교통범죄,소계,우발적,39267,33334,33785,30450,29908,28692,26044,25804,24602,25588,24994,28405
425,소계,교통범죄,소계,현실불만,167,139,308,347,236,299,102,108,70,78,127,103
426,소계,교통범죄,소계,부주의,187583,182010,186010,184435,186037,179206,160586,137464,124880,112601,102147,99616
427,소계,교통범죄,소계,기타,169746,165428,182156,174684,168770,174876,147899,109571,85838,79180,73290,76020


In [25]:
vehicle_motive_df.set_index("범행동기", inplace=True)
vehicle_motive_df.drop(["대", "중", "소"], axis=1, inplace=True)
vehicle_motive_df = vehicle_motive_df.transpose()
vehicle_motive_df

/tmp/ipykernel_19617/2353725930.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



범행동기,이욕,사행심,보복,가정불화,호기심,유혹,우발적,현실불만,부주의,기타,미상
2011년,8251,187,4,233,1490,2082,39267,167,187583,169746,115340
2012년,7834,76,3,174,1535,1767,33334,139,182010,165428,119776
2013년,5529,196,12,209,1587,1293,33785,308,186010,182156,125671
2014년,1315,189,0,192,1387,876,30450,347,184435,174684,139251
2015년,1089,199,3,162,1338,1300,29908,236,186037,168770,160332
2016년,1202,136,3,160,1622,1350,28692,299,179206,174876,167806
2017년,1146,89,2,120,1712,1198,26044,102,160586,147899,145654
2018년,992,233,2,88,606,730,25804,108,137464,109571,137549
2019년,652,255,2,89,701,469,24602,70,124880,85838,146159
2020년,603,254,1,86,1426,488,25588,78,112601,79180,136141


In [28]:
vehicle_motive_df["이욕"][5:]

2016년    1202
2017년    1146
2018년     992
2019년     652
2020년     603
2021년     592
2022년     585
Name: 이욕, dtype: int64

In [ ]:
fig = go.Figure()

fig.update_layout(title_text="2016년도 이후 따른 범죄동기 별 교통범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22 )
fig.update_xaxes(title_text="년도")
fig.update_yaxes(title_text="발생 건수")
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["이욕"][5:], name='이욕', mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["사행심"][5:], name='사행심',mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["보복"][5:], name='보복', mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["가정불화"][5:], name='가정불화', mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["호기심"][5:], name='호기심', mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["유혹"][5:], name='유혹',mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["우발적"][5:], name='우발적', mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["현실불만"][5:], name='현실불만', mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["부주의"][5:], name='부주의',mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["기타"][5:], name='기타', mode='lines+markers'))
fig.add_trace(go.Scatter(x=vehicle_motive_df.index[5:], y=vehicle_motive_df["미상"][5:], name='미상', mode='lines+markers'))

fig.show()

In [8]:
crime.execute("SELECT * FROM crime_time_after")
result = crime.fetchall()

crime_time_df = pd.DataFrame(result)
crime_time_df.head()

,대,중,소,발생시간,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,총범죄,계,소계,00:00~02:59,175531,130371,127974,124068,124560,121805,111172,99089,94095,86398,57763,71808
1,총범죄,계,소계,03:00~05:59,101392,106086,109583,101193,104919,107865,98922,89509,86471,81043,50928,65179
2,총범죄,계,소계,06:00~08:59,88871,94612,95392,93165,102344,108645,97823,87417,88488,81052,68799,69113
3,총범죄,계,소계,09:00~11:59,178310,191682,211569,198855,210771,227039,202798,179038,183235,174325,147157,135849
4,총범죄,계,소계,12:00~14:59,99469,156610,162572,159772,173791,181967,166767,157707,169090,172610,164739,159299


In [81]:
theft_time_df = crime_time_df[crime_time_df["중"] == "절도범죄"]
theft_time_df

,대,중,소,발생시간,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
99,총범죄,절도범죄,소계,00:00~02:59,33013,23761,23674,21567,19591,15516,13734,12581,13088,11778,9358,10252
100,총범죄,절도범죄,소계,03:00~05:59,20502,23392,24309,21195,19544,15827,14362,13717,14167,13661,10610,11681
101,총범죄,절도범죄,소계,06:00~08:59,19069,20292,19715,17918,16573,13871,12491,12473,13094,12380,12020,13489
102,총범죄,절도범죄,소계,09:00~11:59,31812,31732,31528,29081,26273,22842,21376,20412,21501,21210,21357,23397
103,총범죄,절도범죄,소계,12:00~14:59,20846,32648,33167,30903,29383,26339,24533,24515,27167,26695,26178,28486
104,총범죄,절도범죄,소계,15:00~17:59,34307,36720,36569,34679,32602,28529,26865,26631,28553,28619,27544,30883
105,총범죄,절도범죄,소계,18:00~20:59,44176,45653,44081,40322,35746,29447,26609,25655,27229,25781,24162,26343
106,총범죄,절도범죄,소계,21:00~23:59,38819,39745,37964,35015,30550,25121,22137,19962,20568,19245,17908,18882
107,총범죄,절도범죄,소계,미상,38818,36517,37336,35542,35591,25545,21650,20863,21590,20148,17272,18857


In [82]:
theft_time_df = theft_time_df.melt(id_vars=["대","중","소","발생시간"], var_name="년도", value_name="발생건수")

In [83]:
theft_time_df

,대,중,소,발생시간,년도,발생건수
0,총범죄,절도범죄,소계,00:00~02:59,2011년,33013
1,총범죄,절도범죄,소계,03:00~05:59,2011년,20502
2,총범죄,절도범죄,소계,06:00~08:59,2011년,19069
3,총범죄,절도범죄,소계,09:00~11:59,2011년,31812
4,총범죄,절도범죄,소계,12:00~14:59,2011년,20846
...,...,...,...,...,...,...
103,총범죄,절도범죄,소계,12:00~14:59,2022년,28486
104,총범죄,절도범죄,소계,15:00~17:59,2022년,30883
105,총범죄,절도범죄,소계,18:00~20:59,2022년,26343
106,총범죄,절도범죄,소계,21:00~23:59,2022년,18882


In [104]:
fig = px.scatter(theft_time_df, x="년도", y="발생건수", size="발생건수", color="발생시간", log_y=True, size_max=17)
fig.update_layout(title_text="연도별 범죄 시간대 따른 절도범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22 )
fig.show()

In [102]:
fig = px.bar(theft_time_df, x="년도", y="발생건수", color="발생시간")
fig.update_layout(title_text="연도별 범죄 시간대 따른 절도범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22)
fig.show()

In [9]:
vehicle_time_df = crime_time_df[crime_time_df["중"] == "교통범죄"]
vehicle_time_df

,대,중,소,발생시간,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
342,총범죄,교통범죄,소계,00:00~02:59,56475,42507,41457,41939,42233,40532,36241,29333,24513,23344,15052,18498
343,총범죄,교통범죄,소계,03:00~05:59,30616,31839,31388,32441,35125,37506,33054,27028,23438,21560,12875,15933
344,총범죄,교통범죄,소계,06:00~08:59,39598,41865,42961,44610,49475,54513,45162,37387,37189,31329,24317,18566
345,총범죄,교통범죄,소계,09:00~11:59,43072,44621,49310,53045,59046,66856,53784,42453,44265,39356,33579,21755
346,총범죄,교통범죄,소계,12:00~14:59,33404,47385,50490,54231,60036,63468,51906,42866,45675,44060,39711,23858
347,총범죄,교통범죄,소계,15:00~17:59,58581,59278,62000,65612,73760,76556,64011,52436,53506,51132,46110,28128
348,총범죄,교통범죄,소계,18:00~20:59,70419,75466,76543,79167,83198,84797,71345,60050,58164,56373,52167,35737
349,총범죄,교통범죄,소계,21:00~23:59,160684,157988,173041,157320,152143,138810,112998,90043,70537,63335,69859,59900
350,총범죄,교통범죄,소계,미상,43318,41835,46263,45128,41649,37363,32661,26775,20067,18236,14964,18654


In [10]:
vehicle_time_df = vehicle_time_df.melt(id_vars=["대", "중", "소", "발생시간"], var_name="년도", value_name="발생건수")
vehicle_time_df

,대,중,소,발생시간,년도,발생건수
0,총범죄,교통범죄,소계,00:00~02:59,2011년,56475
1,총범죄,교통범죄,소계,03:00~05:59,2011년,30616
2,총범죄,교통범죄,소계,06:00~08:59,2011년,39598
3,총범죄,교통범죄,소계,09:00~11:59,2011년,43072
4,총범죄,교통범죄,소계,12:00~14:59,2011년,33404
...,...,...,...,...,...,...
103,총범죄,교통범죄,소계,12:00~14:59,2022년,23858
104,총범죄,교통범죄,소계,15:00~17:59,2022년,28128
105,총범죄,교통범죄,소계,18:00~20:59,2022년,35737
106,총범죄,교통범죄,소계,21:00~23:59,2022년,59900


In [12]:
vehicle_time_df = vehicle_time_df[( vehicle_time_df["년도"] == "2016년" ) | ( vehicle_time_df["년도"] == "2017년" ) | ( vehicle_time_df["년도"] == "2018년" ) | ( vehicle_time_df["년도"] == "2019년" )| ( vehicle_time_df["년도"] == "2020년" )| ( vehicle_time_df["년도"] == "2021년" )| ( vehicle_time_df["년도"] == "2022년" ) ]
vehicle_time_df

,대,중,소,발생시간,년도,발생건수
45,총범죄,교통범죄,소계,00:00~02:59,2016년,40532
46,총범죄,교통범죄,소계,03:00~05:59,2016년,37506
47,총범죄,교통범죄,소계,06:00~08:59,2016년,54513
48,총범죄,교통범죄,소계,09:00~11:59,2016년,66856
49,총범죄,교통범죄,소계,12:00~14:59,2016년,63468
...,...,...,...,...,...,...
103,총범죄,교통범죄,소계,12:00~14:59,2022년,23858
104,총범죄,교통범죄,소계,15:00~17:59,2022년,28128
105,총범죄,교통범죄,소계,18:00~20:59,2022년,35737
106,총범죄,교통범죄,소계,21:00~23:59,2022년,59900


In [14]:
fig = px.bar(vehicle_time_df, x="년도", y="발생건수", color="발생시간")
fig.update_layout(title_text="2016년 이후 시간대 따른 교통범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22)
fig.show()

In [5]:
crime.execute("SELECT * FROM crime_dotweek_after")
result = crime.fetchall()

crime_dotweek_df = pd.DataFrame(result)
crime_dotweek_df.head()

,대,중,소,발생요일,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,총범죄,강력범죄,강간,금요일,0,0,819,746,699,760,757,699,722,760,695,772
1,총범죄,강력범죄,강간,목요일,0,0,767,680,629,615,665,646,682,701,706,681
2,총범죄,강력범죄,강간,수요일,0,0,711,673,683,655,680,743,719,712,654,718
3,총범죄,강력범죄,강간,월요일,0,0,759,628,670,655,690,698,686,699,773,770
4,총범죄,강력범죄,강간,일요일,0,0,925,789,875,873,890,906,869,909,878,869


In [6]:
theft_dotweek_df = crime_dotweek_df[ crime_dotweek_df["중"] == "절도범죄"]
theft_dotweek_df

,대,중,소,발생요일,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
133,총범죄,절도범죄,소계,금요일,42692,44051,43634,40471,37363,30852,27505,26398,28156,27311,25209,28133
134,총범죄,절도범죄,소계,목요일,39233,40576,40372,36805,33668,27809,25746,24400,26098,24974,23989,26137
135,총범죄,절도범죄,소계,수요일,39674,40306,40451,36689,34908,27861,25147,24761,26061,25538,23743,26203
136,총범죄,절도범죄,소계,월요일,39466,39237,40199,35699,34039,27683,25011,24354,26239,25196,23429,25210
137,총범죄,절도범죄,소계,일요일,37476,40168,38781,36895,33217,28759,25926,24832,25466,23771,21687,23632
138,총범죄,절도범죄,소계,토요일,43084,46192,45667,42693,39215,32445,29623,27786,29266,27557,24897,27511
139,총범죄,절도범죄,소계,화요일,39737,39930,39239,36970,33443,27628,24799,24278,25671,25170,23455,25444


In [7]:
theft_dotweek_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 133 to 139
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   대       7 non-null      object
 1   중       7 non-null      object
 2   소       7 non-null      object
 3   발생요일    7 non-null      object
 4   2011년   7 non-null      int64 
 5   2012년   7 non-null      int64 
 6   2013년   7 non-null      int64 
 7   2014년   7 non-null      int64 
 8   2015년   7 non-null      int64 
 9   2016년   7 non-null      int64 
 10  2017년   7 non-null      int64 
 11  2018년   7 non-null      int64 
 12  2019년   7 non-null      int64 
 13  2020년   7 non-null      int64 
 14  2021년   7 non-null      int64 
 15  2022년   7 non-null      int64 
dtypes: int64(12), object(4)
memory usage: 952.0+ bytes


In [8]:
theft_dotweek_df = theft_dotweek_df.melt(id_vars=["대", "중", "소", "발생요일"], var_name="년도", value_name="발생건수" )
theft_dotweek_df

,대,중,소,발생요일,년도,발생건수
0,총범죄,절도범죄,소계,금요일,2011년,42692
1,총범죄,절도범죄,소계,목요일,2011년,39233
2,총범죄,절도범죄,소계,수요일,2011년,39674
3,총범죄,절도범죄,소계,월요일,2011년,39466
4,총범죄,절도범죄,소계,일요일,2011년,37476
...,...,...,...,...,...,...
79,총범죄,절도범죄,소계,수요일,2022년,26203
80,총범죄,절도범죄,소계,월요일,2022년,25210
81,총범죄,절도범죄,소계,일요일,2022년,23632
82,총범죄,절도범죄,소계,토요일,2022년,27511


In [12]:
fig = px.line(theft_dotweek_df, x="년도", y="발생건수", color="발생요일", markers=True, range_y=[0, 60000])
fig.update_layout(title_text="년도 별 요일에 따른 절도범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22)
fig.show()

In [22]:
vehicle_dotweek_df = crime_dotweek_df[crime_dotweek_df["중"] == "교통범죄"]
vehicle_dotweek_df

,대,중,소,발생요일,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
70,총범죄,교통범죄,소계,금요일,81465,84446,90658,89157,92973,91267,76963,63750,58103,54381,48221,38316
71,총범죄,교통범죄,소계,목요일,78133,78722,82147,82368,85819,87438,73767,59736,54846,50518,45010,34872
72,총범죄,교통범죄,소계,수요일,76779,76301,81327,81707,84795,85277,71558,58184,54509,49712,44706,34202
73,총범죄,교통범죄,소계,월요일,70789,70692,75153,76089,79700,80279,66657,54328,51459,48515,42843,31335
74,총범죄,교통범죄,소계,일요일,70572,71365,75094,74665,77266,79265,65739,52166,47698,43196,37318,31817
75,총범죄,교통범죄,소계,토요일,83000,84792,89380,88756,91183,91712,76545,62725,57134,52071,46324,36943
76,총범죄,교통범죄,소계,화요일,75429,76466,79694,80751,84929,85163,69933,57482,53605,50332,44212,33544


In [24]:
vehicle_dotweek_df = vehicle_dotweek_df.melt(id_vars=["대", "중", "소", "발생요일"], var_name="년도", value_name="발생건수")
vehicle_dotweek_df

,대,중,소,발생요일,년도,발생건수
0,총범죄,교통범죄,소계,금요일,2011년,81465
1,총범죄,교통범죄,소계,목요일,2011년,78133
2,총범죄,교통범죄,소계,수요일,2011년,76779
3,총범죄,교통범죄,소계,월요일,2011년,70789
4,총범죄,교통범죄,소계,일요일,2011년,70572
...,...,...,...,...,...,...
79,총범죄,교통범죄,소계,수요일,2022년,34202
80,총범죄,교통범죄,소계,월요일,2022년,31335
81,총범죄,교통범죄,소계,일요일,2022년,31817
82,총범죄,교통범죄,소계,토요일,2022년,36943


In [25]:
vehicle_dotweek_df = vehicle_dotweek_df[( vehicle_dotweek_df["년도"] == "2016년" ) | ( vehicle_dotweek_df["년도"] == "2017년" ) | ( vehicle_dotweek_df["년도"] == "2018년" ) | ( vehicle_dotweek_df["년도"] == "2019년" )| ( vehicle_dotweek_df["년도"] == "2020년" )| ( vehicle_dotweek_df["년도"] == "2021년" )| ( vehicle_dotweek_df["년도"] == "2022년" ) ]
vehicle_dotweek_df

,대,중,소,발생요일,년도,발생건수
35,총범죄,교통범죄,소계,금요일,2016년,91267
36,총범죄,교통범죄,소계,목요일,2016년,87438
37,총범죄,교통범죄,소계,수요일,2016년,85277
38,총범죄,교통범죄,소계,월요일,2016년,80279
39,총범죄,교통범죄,소계,일요일,2016년,79265
40,총범죄,교통범죄,소계,토요일,2016년,91712
41,총범죄,교통범죄,소계,화요일,2016년,85163
42,총범죄,교통범죄,소계,금요일,2017년,76963
43,총범죄,교통범죄,소계,목요일,2017년,73767
44,총범죄,교통범죄,소계,수요일,2017년,71558


In [44]:
fig = px.bar(vehicle_dotweek_df, x="발생요일", y="발생건수", color="발생요일", animation_frame="년도", text="발생건수" )
fig.update_layout(title_text="2016년도 이후 요일에 따른 교통범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22)
fig.show()

In [46]:
crime.execute("SELECT * FROM crime_place_after")
result = crime.fetchall()

crime_place_df = pd.DataFrame(result)
crime_place_df.head()

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,총범죄,소계,소계,아파트 연립다세대,59344,52509,77667,60265,62187,63578,58288,82841,96879,100386,106001,108301
1,총범죄,소계,소계,단독주택,85371,77671,106024,79371,82428,74128,64757,81199,84895,79936,71347,69742
2,총범죄,소계,소계,고속도로,4942,4254,3659,4305,4481,5111,6992,6714,7762,7358,8152,10233
3,총범죄,소계,소계,노상,709011,689785,827395,786803,814101,840857,726140,644537,625533,603582,551630,501535
4,총범죄,소계,소계,백화점,2055,1987,3146,2480,2534,2614,2466,3546,4155,3471,2808,3047


In [50]:
theft_place_df = crime_place_df[crime_place_df["중"] == "절도범죄"]

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
374,총범죄,절도범죄,소계,아파트 연립다세대,19657,14800,22706,14333,11042,8774,7036,11515,14344,16697,16482,17196
375,총범죄,절도범죄,소계,단독주택,26411,20401,30221,19515,15565,13226,9044,11542,11448,10196,8535,8117
376,총범죄,절도범죄,소계,고속도로,122,128,179,177,133,144,144,164,181,161,99,115
377,총범죄,절도범죄,소계,노상,46213,40448,65598,49527,45307,39355,36798,39952,45539,46861,46374,51945
378,총범죄,절도범죄,소계,백화점,1500,1493,2409,1828,1928,1990,1816,2599,3187,2695,2142,2274
379,총범죄,절도범죄,소계,슈퍼마켓,3875,3475,6400,4718,5080,4890,4571,6279,7673,9273,9920,11703
380,총범죄,절도범죄,소계,편의점,2952,2975,5083,4579,4351,4151,4160,5169,5853,5944,6143,6994
381,총범죄,절도범죄,소계,대형할인매장,2947,2757,2992,1966,2170,2104,1767,2413,2693,3065,3350,3536
382,총범죄,절도범죄,소계,상점,11289,9568,16865,9520,8850,9051,7787,13107,14506,13629,15031,17550
383,총범죄,절도범죄,소계,시장노점,3094,3131,5237,2998,2475,2311,1803,4099,4677,3951,3970,3927


In [53]:
theft_place_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   대       408 non-null    object
 1   중       408 non-null    object
 2   소       408 non-null    object
 3   발생장소    408 non-null    object
 4   년도      408 non-null    object
 5   발생건수    408 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 19.2+ KB


In [52]:
theft_place_df = theft_place_df.melt(id_vars=["대", "중", "소", "발생장소"], var_name="년도", value_name="발생건수")

ValueError: value_name (발생건수) cannot match an element in the DataFrame columns.

In [54]:
theft_place_df

,대,중,소,발생장소,년도,발생건수
0,총범죄,절도범죄,소계,아파트 연립다세대,2011년,19657
1,총범죄,절도범죄,소계,단독주택,2011년,26411
2,총범죄,절도범죄,소계,고속도로,2011년,122
3,총범죄,절도범죄,소계,노상,2011년,46213
4,총범죄,절도범죄,소계,백화점,2011년,1500
...,...,...,...,...,...,...
403,총범죄,절도범죄,소계,공지,2022년,52
404,총범죄,절도범죄,소계,주차장,2022년,10381
405,총범죄,절도범죄,소계,공중화장실,2022년,703
406,총범죄,절도범죄,소계,PC방,2022년,994


In [57]:
fig = px.line(theft_place_df, x="년도", y="발생건수", color="발생장소", markers=True )
fig.update_layout(title_text="년도 별 발생장소에 따른 절도범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22)
fig.show()

In [58]:
vehicle_place_df = crime_place_df[crime_place_df["중"]=="교통범죄"]
vehicle_place_df

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
1292,총범죄,교통범죄,소계,아파트 연립다세대,134,80,134,105,124,96,80,101,104,114,105,106
1293,총범죄,교통범죄,소계,단독주택,40,18,33,34,41,40,36,34,27,45,38,37
1294,총범죄,교통범죄,소계,고속도로,2133,3052,2519,2578,2727,3516,4881,4408,5163,4154,5327,7471
1295,총범죄,교통범죄,소계,노상,524351,526938,567776,562472,584632,587582,488787,401095,369771,342058,300530,230183
1296,총범죄,교통범죄,소계,백화점,0,2,2,1,0,1,0,1,0,0,1,0
1297,총범죄,교통범죄,소계,슈퍼마켓,6,3,4,3,3,5,3,2,2,1,1,2
1298,총범죄,교통범죄,소계,편의점,2,3,1,1,5,5,3,3,6,4,4,3
1299,총범죄,교통범죄,소계,대형할인매장,2,1,1,1,0,0,0,3,2,1,0,0
1300,총범죄,교통범죄,소계,상점,42,32,58,38,45,39,30,32,42,29,36,48
1301,총범죄,교통범죄,소계,시장노점,62,40,44,55,102,80,55,103,91,59,79,53


In [59]:
vehicle_place_df = vehicle_place_df.melt(id_vars=["대", "중", "소", "발생장소"], var_name="년도", value_name="발생건수")
vehicle_place_df

,대,중,소,발생장소,년도,발생건수
0,총범죄,교통범죄,소계,아파트 연립다세대,2011년,134
1,총범죄,교통범죄,소계,단독주택,2011년,40
2,총범죄,교통범죄,소계,고속도로,2011년,2133
3,총범죄,교통범죄,소계,노상,2011년,524351
4,총범죄,교통범죄,소계,백화점,2011년,0
...,...,...,...,...,...,...
403,총범죄,교통범죄,소계,공지,2022년,3
404,총범죄,교통범죄,소계,주차장,2022년,692
405,총범죄,교통범죄,소계,공중화장실,2022년,0
406,총범죄,교통범죄,소계,PC방,2022년,0


In [60]:
vehicle_place_df = vehicle_place_df[( vehicle_place_df["년도"] == "2016년" ) | ( vehicle_place_df["년도"] == "2017년" ) | ( vehicle_place_df["년도"] == "2018년" ) | ( vehicle_place_df["년도"] == "2019년" )| ( vehicle_place_df["년도"] == "2020년" )| ( vehicle_place_df["년도"] == "2021년" )| ( vehicle_place_df["년도"] == "2022년" ) ]
vehicle_place_df

,대,중,소,발생장소,년도,발생건수
170,총범죄,교통범죄,소계,아파트 연립다세대,2016년,96
171,총범죄,교통범죄,소계,단독주택,2016년,40
172,총범죄,교통범죄,소계,고속도로,2016년,3516
173,총범죄,교통범죄,소계,노상,2016년,587582
174,총범죄,교통범죄,소계,백화점,2016년,1
...,...,...,...,...,...,...
403,총범죄,교통범죄,소계,공지,2022년,3
404,총범죄,교통범죄,소계,주차장,2022년,692
405,총범죄,교통범죄,소계,공중화장실,2022년,0
406,총범죄,교통범죄,소계,PC방,2022년,0


In [62]:
fig = px.line(vehicle_place_df, x="년도", y="발생건수", color="발생장소", markers=True )
fig.update_layout(title_text="2016년 이후 발생장소에 따른 교통범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22)
fig.show()

In [42]:
crime.execute("SELECT * FROM crime_dosi_after")
result = crime.fetchall()

crime_dosi_df = pd.DataFrame(result)
crime_dosi_df.head()

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,총범죄,소계,소계,서울,358753,360468,368876,355430,355341,341925,319046,306661,305909,290816,252289,267778
1,총범죄,소계,소계,부산,135399,137788,140078,135757,143713,133966,118854,115727,119267,120095,105737,112080
2,총범죄,소계,소계,대구,96450,97968,96254,95802,98758,86094,75024,72270,75707,72171,67795,66441
3,총범죄,소계,소계,인천,86641,92766,99056,96846,101931,103459,95244,88228,93347,91257,80589,84888
4,총범죄,소계,소계,광주,63810,67513,68034,64748,61697,54287,48205,43515,44102,43179,39203,38408


In [43]:
theft_dosi_df = crime_dosi_df[crime_dosi_df["중"] == "절도범죄"]
theft_dosi_df

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
935,총범죄,절도범죄,소계,서울,54310,61332,61600,59371,55280,46861,41144,39182,42224,38296,33529,37594
936,총범죄,절도범죄,소계,부산,24697,25494,23887,21898,20136,16777,15807,14373,14497,14189,12667,13699
937,총범죄,절도범죄,소계,대구,14852,16554,17481,14609,12759,9171,8310,8169,9249,8675,9306,9792
938,총범죄,절도범죄,소계,인천,10146,10394,10989,9737,10679,10025,9216,9473,10134,8991,8175,8895
939,총범죄,절도범죄,소계,광주,13373,12678,12419,10361,8438,6050,4821,4832,4922,4939,4884,5204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,총범죄,절도범죄,소계,통영시,0,0,0,0,602,551,480,412,466,478,412,387
1019,총범죄,절도범죄,소계,서귀포시,0,0,0,0,921,927,864,858,907,1155,860,886
1020,총범죄,절도범죄,소계,제주시,5405,5053,5834,4980,3309,2573,2359,2124,2325,2270,2194,2247
1021,총범죄,절도범죄,소계,기타도시,17440,18734,18149,17551,643,369,19,49,31,24,38,43


In [44]:
theft_dosi_df = theft_dosi_df.melt(id_vars=["대", "중", "소", "발생장소"], var_name="년도", value_name="발생건수")
theft_dosi_df

,대,중,소,발생장소,년도,발생건수
0,총범죄,절도범죄,소계,서울,2011년,54310
1,총범죄,절도범죄,소계,부산,2011년,24697
2,총범죄,절도범죄,소계,대구,2011년,14852
3,총범죄,절도범죄,소계,인천,2011년,10146
4,총범죄,절도범죄,소계,광주,2011년,13373
...,...,...,...,...,...,...
1051,총범죄,절도범죄,소계,통영시,2022년,387
1052,총범죄,절도범죄,소계,서귀포시,2022년,886
1053,총범죄,절도범죄,소계,제주시,2022년,2247
1054,총범죄,절도범죄,소계,기타도시,2022년,43


In [46]:
theft_dosi_df["발생장소"].unique()

array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '고양시', '과천시',
       '광명시', '광주시', '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남시',
       '수원시', '시흥시', '안산시', '안성시', '안양시', '양주시', '여주시', '오산시', '용인시',
       '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시', '강릉시',
       '동해시', '삼척시', '속초시', '원주시', '춘천시', '태백시', '제천시', '청주시', '충주시',
       '계룡시', '공주시', '논산시', '당진시', '보령시', '서산시', '아산시', '천안시', '군산시',
       '김제시', '남원시', '익산시', '전주시', '정읍시', '광양시', '나주시', '목포시', '순천시',
       '여수시', '경산시', '경주시', '구미시', '김천시', '문경시', '상주시', '안동시', '영주시',
       '영천시', '포항시', '거제시', '김해시', '마산시', '밀양시', '사천시', '양산시', '진주시',
       '창원시', '통영시', '서귀포시', '제주시', '기타도시', '도시이외'], dtype=object)

In [69]:
fig = px.line(theft_dosi_df, x="년도", y="발생건수", color="발생장소", markers=True )
fig.update_layout(title_text="년도별 발생 지역에 따른 절도범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22)
fig.show()

In [70]:
vehicle_sido_df = crime_dosi_df[crime_dosi_df["중"]== "교통범죄"]
vehicle_sido_df

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
3311,총범죄,교통범죄,소계,서울,87467,79627,86560,76497,74958,74270,67430,61132,55273,50895,43206,32013
3312,총범죄,교통범죄,소계,부산,32131,32758,31638,36035,35841,32944,28061,22894,20433,18201,16160,12176
3313,총범죄,교통범죄,소계,대구,34209,36180,35431,38676,39725,31682,25380,22200,21012,19005,16619,12610
3314,총범죄,교통범죄,소계,인천,28086,30491,30232,32322,31163,30972,24507,18173,17749,16771,14459,11926
3315,총범죄,교통범죄,소계,광주,21878,24387,26251,24008,23412,22137,18734,13779,12747,12432,11121,7827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3394,총범죄,교통범죄,소계,통영시,0,0,0,0,2936,2430,2031,1350,1262,1245,1019,825
3395,총범죄,교통범죄,소계,서귀포시,0,0,0,0,3581,4060,3713,2378,1895,1904,1916,1382
3396,총범죄,교통범죄,소계,제주시,7422,9328,11848,12953,10857,11642,9936,6417,5147,4632,4266,3052
3397,총범죄,교통범죄,소계,기타도시,45967,47685,53138,55076,2325,1287,239,319,184,129,204,323


In [71]:
vehicle_sido_df = vehicle_sido_df.melt(id_vars=["대","중","소","발생장소"], var_name="년도", value_name="발생건수")
vehicle_sido_df

,대,중,소,발생장소,년도,발생건수
0,총범죄,교통범죄,소계,서울,2011년,87467
1,총범죄,교통범죄,소계,부산,2011년,32131
2,총범죄,교통범죄,소계,대구,2011년,34209
3,총범죄,교통범죄,소계,인천,2011년,28086
4,총범죄,교통범죄,소계,광주,2011년,21878
...,...,...,...,...,...,...
1051,총범죄,교통범죄,소계,통영시,2022년,825
1052,총범죄,교통범죄,소계,서귀포시,2022년,1382
1053,총범죄,교통범죄,소계,제주시,2022년,3052
1054,총범죄,교통범죄,소계,기타도시,2022년,323


In [72]:
vehicle_sido_df = vehicle_sido_df[( vehicle_sido_df["년도"] == "2016년" ) | ( vehicle_sido_df["년도"] == "2017년" ) | ( vehicle_sido_df["년도"] == "2018년" ) | ( vehicle_sido_df["년도"] == "2019년" )| ( vehicle_sido_df["년도"] == "2020년" )| ( vehicle_sido_df["년도"] == "2021년" )| ( vehicle_sido_df["년도"] == "2022년" ) ]
vehicle_sido_df

,대,중,소,발생장소,년도,발생건수
440,총범죄,교통범죄,소계,서울,2016년,74270
441,총범죄,교통범죄,소계,부산,2016년,32944
442,총범죄,교통범죄,소계,대구,2016년,31682
443,총범죄,교통범죄,소계,인천,2016년,30972
444,총범죄,교통범죄,소계,광주,2016년,22137
...,...,...,...,...,...,...
1051,총범죄,교통범죄,소계,통영시,2022년,825
1052,총범죄,교통범죄,소계,서귀포시,2022년,1382
1053,총범죄,교통범죄,소계,제주시,2022년,3052
1054,총범죄,교통범죄,소계,기타도시,2022년,323


In [108]:
fig = px.line(vehicle_sido_df, x="년도", y="발생건수", color="발생장소", markers=True )
fig.update_layout(title_text="년도별 발생 지역에 따른 교통범죄 발생 건 수", xaxis_tickangle=-45, title_x = 0.5, title_font_size=22)
fig.show()

In [14]:
ub3c4
crime_count_df = pd.DataFrame(result)
crime_count_df.head()

,대,중,소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,총계,소계,소계,1752598,1793400,1857276,1778966,1861657,1849450,1662341,1580751,1611906,1587866,1429826,1482433
1,총계,강력범죄,소계,26699,25152,26962,25277,25334,25765,27274,26787,26476,24332,22476,24954
2,총계,강력범죄,살인기수,427,411,342,372,365,356,301,309,297,308,270,275
3,총계,강력범죄,살인미수등,777,584,587,541,564,558,524,488,482,416,388,427
4,총계,강력범죄,강도,3994,2587,1980,1586,1446,1149,967,821,798,663,495,516


In [15]:
theft_count_df = crime_count_df[ crime_count_df["중"] == "절도범죄" ]
theft_count_df

,대,중,소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
11,총계,절도범죄,소계,281362,290460,288343,266222,245853,203037,183757,176809,186957,179517,166409,182270
12,총계,절도범죄,절도,241401,253166,248785,229194,211886,175765,160627,155723,165358,158512,148196,164143
13,총계,절도범죄,절도(상습),0,0,1052,805,5017,3123,1940,1218,1070,971,785,697
14,총계,절도범죄,특수절도,21730,21705,20362,17921,18636,15506,13128,11955,11783,11772,9142,9411
15,총계,절도범죄,특수절도(상습),40,28,14,46,445,449,87,56,81,79,23,56
16,총계,절도범죄,야간주거침입절도,5018,4553,5537,6292,7439,7065,6659,5994,6329,5465,5153,4697
17,총계,절도범죄,야간주거침입절도(상습),16,14,15,20,333,276,132,89,122,105,109,32
18,총계,절도범죄,불법사용,303,294,320,359,380,518,542,577,648,830,802,870
19,총계,절도범죄,불법사용(상습),0,0,0,0,0,1,1,0,2,0,1,0
20,총계,절도범죄,절도·불법사용(상습),599,365,0,0,0,0,0,0,0,0,0,0


In [16]:
theft_count_df = theft_count_df.melt(id_vars = ["대", "중", "소"], var_name="년도", value_name="발생건수")
theft_count_df

,대,중,소,년도,발생건수
0,총계,절도범죄,소계,2011년,281362
1,총계,절도범죄,절도,2011년,241401
2,총계,절도범죄,절도(상습),2011년,0
3,총계,절도범죄,특수절도,2011년,21730
4,총계,절도범죄,특수절도(상습),2011년,40
...,...,...,...,...,...
139,총계,절도범죄,불법사용,2022년,870
140,총계,절도범죄,불법사용(상습),2022년,0
141,총계,절도범죄,절도·불법사용(상습),2022년,0
142,총계,절도범죄,특가법(절도),2022년,2364


In [17]:
theft_count_total_df = theft_count_df[theft_count_df["소"]== "소계"]
theft_count_total_df

,대,중,소,년도,발생건수
0,총계,절도범죄,소계,2011년,281362
12,총계,절도범죄,소계,2012년,290460
24,총계,절도범죄,소계,2013년,288343
36,총계,절도범죄,소계,2014년,266222
48,총계,절도범죄,소계,2015년,245853
60,총계,절도범죄,소계,2016년,203037
72,총계,절도범죄,소계,2017년,183757
84,총계,절도범죄,소계,2018년,176809
96,총계,절도범죄,소계,2019년,186957
108,총계,절도범죄,소계,2020년,179517


In [18]:
theft_count_df = theft_count_df[theft_count_df["소"] != "소계"]
theft_count_df

,대,중,소,년도,발생건수
1,총계,절도범죄,절도,2011년,241401
2,총계,절도범죄,절도(상습),2011년,0
3,총계,절도범죄,특수절도,2011년,21730
4,총계,절도범죄,특수절도(상습),2011년,40
5,총계,절도범죄,야간주거침입절도,2011년,5018
...,...,...,...,...,...
139,총계,절도범죄,불법사용,2022년,870
140,총계,절도범죄,불법사용(상습),2022년,0
141,총계,절도범죄,절도·불법사용(상습),2022년,0
142,총계,절도범죄,특가법(절도),2022년,2364


In [19]:
theft_count_df["소"].unique()

array(['절도', '절도(상습)', '특수절도', '특수절도(상습)', '야간주거침입절도', '야간주거침입절도(상습)',
       '불법사용', '불법사용(상습)', '절도·불법사용(상습)', '특가법(절도)', '특가법(단체등의조직)'],
      dtype=object)

In [20]:
fig = make_subplots(2,1, subplot_titles=("<b>연도별 절도범죄 전체 건수 변화</b>", "<b>연도별 절도범죄 세부사항 건수 변화</b>"), print_grid=True)
fig.update_layout(height=800, width=1000, legend=dict(orientation='h', xanchor='left') )

fig.add_trace(go.Scatter(x=theft_count_total_df["년도"], y=theft_count_total_df["발생건수"], mode="markers+lines", name="절도범죄 총 건수"), row=1, col=1)


theft_detail_list = ['절도', '절도(상습)', '특수절도', '특수절도(상습)', '야간주거침입절도', '야간주거침입절도(상습)',
       '불법사용', '불법사용(상습)', '절도·불법사용(상습)', '특가법(절도)', '특가법(단체등의조직)']

for theft_detail in theft_detail_list:
       df = theft_count_df[theft_count_df["소"] == theft_detail]
       fig.add_trace(go.Scatter(x=df["년도"], y=df["발생건수"], mode="markers+lines", name=theft_detail), row=2, col=1)
                 
fig.show()

This is the format of your plot grid:
[ (1,1) x,y   ]
[ (2,1) x2,y2 ]



In [21]:
vehicle_count_df = crime_count_df[crime_count_df["중"] == "교통범죄"]
vehicle_count_df

,대,중,소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
100,총계,교통범죄,소계,536167,542784,573453,573493,596665,600401,501162,408371,377354,348725,308634,241029
101,총계,교통범죄,교통사고,0,297226,298083,313617,338212,324770,269434,237354,240802,225762,190488,109931
102,총계,교통범죄,교통사고외,0,245558,275370,259876,258453,275631,231728,171017,136552,122963,118146,131098


In [22]:
vehicle_count_df = vehicle_count_df.melt(id_vars=["대", "중", "소"], var_name="년도", value_name="발생건수")
vehicle_count_df

,대,중,소,년도,발생건수
0,총계,교통범죄,소계,2011년,536167
1,총계,교통범죄,교통사고,2011년,0
2,총계,교통범죄,교통사고외,2011년,0
3,총계,교통범죄,소계,2012년,542784
4,총계,교통범죄,교통사고,2012년,297226
5,총계,교통범죄,교통사고외,2012년,245558
6,총계,교통범죄,소계,2013년,573453
7,총계,교통범죄,교통사고,2013년,298083
8,총계,교통범죄,교통사고외,2013년,275370
9,총계,교통범죄,소계,2014년,573493


In [23]:
vehicle_count_df = vehicle_count_df[( vehicle_count_df["년도"] == "2016년" ) | ( vehicle_count_df["년도"] == "2017년" ) | ( vehicle_count_df["년도"] == "2018년" ) | ( vehicle_count_df["년도"] == "2019년" )| ( vehicle_count_df["년도"] == "2020년" )| ( vehicle_count_df["년도"] == "2021년" )| ( vehicle_count_df["년도"] == "2022년" ) ]
vehicle_count_df

,대,중,소,년도,발생건수
15,총계,교통범죄,소계,2016년,600401
16,총계,교통범죄,교통사고,2016년,324770
17,총계,교통범죄,교통사고외,2016년,275631
18,총계,교통범죄,소계,2017년,501162
19,총계,교통범죄,교통사고,2017년,269434
20,총계,교통범죄,교통사고외,2017년,231728
21,총계,교통범죄,소계,2018년,408371
22,총계,교통범죄,교통사고,2018년,237354
23,총계,교통범죄,교통사고외,2018년,171017
24,총계,교통범죄,소계,2019년,377354


In [24]:
vehicle_count_total_df = vehicle_count_df[vehicle_count_df["소"] == "소계"]
vehicle_count_total_df

,대,중,소,년도,발생건수
15,총계,교통범죄,소계,2016년,600401
18,총계,교통범죄,소계,2017년,501162
21,총계,교통범죄,소계,2018년,408371
24,총계,교통범죄,소계,2019년,377354
27,총계,교통범죄,소계,2020년,348725
30,총계,교통범죄,소계,2021년,308634
33,총계,교통범죄,소계,2022년,241029


In [25]:
vehicle_count_df = vehicle_count_df[vehicle_count_df["소"] != "소계"]
vehicle_count_df

,대,중,소,년도,발생건수
16,총계,교통범죄,교통사고,2016년,324770
17,총계,교통범죄,교통사고외,2016년,275631
19,총계,교통범죄,교통사고,2017년,269434
20,총계,교통범죄,교통사고외,2017년,231728
22,총계,교통범죄,교통사고,2018년,237354
23,총계,교통범죄,교통사고외,2018년,171017
25,총계,교통범죄,교통사고,2019년,240802
26,총계,교통범죄,교통사고외,2019년,136552
28,총계,교통범죄,교통사고,2020년,225762
29,총계,교통범죄,교통사고외,2020년,122963


In [39]:
fig = make_subplots(2,1, subplot_titles=("<b>연도별 교통범죄 전체 건수 변화</b>", "<b>연도별 교통범죄 세부사항 건수 변화</b>"), print_grid=True)
fig.update_layout(height=800, width=1100, legend=dict(orientation='v', xanchor='center', yanchor="middle") )

fig.add_trace(go.Scatter(x=vehicle_count_total_df["년도"], y=vehicle_count_total_df["발생건수"], mode="markers+lines", name="교통범죄 총 건수"), row=1, col=1)

vehicle_detail_list = ["교통사고", "교통사고외"]

for vehicle_detail in vehicle_detail_list:
       df = vehicle_count_df[vehicle_count_df["소"] == vehicle_detail]
       fig.add_trace(go.Bar(x=df["년도"], y=df["발생건수"], name=vehicle_detail, text=df["발생건수"], textposition="inside"), row=2, col=1)
                 
fig.show()

This is the format of your plot grid:
[ (1,1) x,y   ]
[ (2,1) x2,y2 ]

